### Setup 
Importing needed liraries and modules

In [2]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

# import functions from data processing module
from process_data import get_votes_from_country
from process_data import get_migrants_from_country
from process_data import clean_data
from process_data import extract_participants

Here we have the dataframe obtained from the eurovision_winners file, where we have displayed the winner country for every year and the language of the song with wich this country have winned the contest.

In [21]:
winners_data = pd.read_csv('eurovision_winners.csv')

winners_data.head()

,Year,Winner,SongLanguage
0,1990,Italy,italian
1,1991,Sweden,swedish
2,1992,Ireland,english
3,1993,Ireland,english
4,1994,Ireland,english


We can sample the random country (along with the relevant data for this country)

But we have pre-chosen one country from the winners_data to make the ibservations clearer and easier to work with.

In [8]:
# random sampling of the year the winner of this year (some country)
sample = winners_data.sample()
YEAR = sample.iat[0,0]
COUNTRY = sample.iat[0,1]
LANGUAGE = sample.iat[0,2]

In [9]:
# example
YEAR = '2015'
COUNTRY = 'Sweden'
LANGUAGE = 'english'

In the following chunk we are:
- getting the data about points given to the chosen country from the other countries in the given year.
- votes_this are the votes given in this year and the votes_previous are the votes given in the previous year
- after that we are combining these two obtained series into one dataframe

In [10]:
# get votes for COUNTRY in the chosen YEAR
this_year = get_votes_from_country(str(COUNTRY.lower()), 'televoters', str(YEAR))

votes_this = pd.DataFrame(list(this_year.items()), \
                    index = [x for x in range(len(this_year.items()))], \
                    columns = ['country', 'votes_this'])
votes_this['votes_this'] = votes_this['votes_this'].astype(int)

# get votes for COUNTRY in the previous year (YEAR - 1)
previous = get_votes_from_country(str(COUNTRY.lower()), 'televoters', str(int(YEAR) - 1))

votes_previous = pd.DataFrame(list(previous.items()), \
                    index = [x for x in range(len(previous.items()))], \
                    columns = ['country', 'votes_previous'])
votes_previous['votes_previous'] = votes_previous['votes_previous'].astype(int)

# join series into one dataframe with votes
votes = pd.concat([votes_this, votes_previous])
votes = votes.groupby(['country']).sum().reset_index()

votes.head()

,country,votes_this,votes_previous
0,albania,7.0,0.0
1,armenia,7.0,0.0
2,australia,12.0,0.0
3,austria,7.0,6.0
4,azerbaijan,6.0,0.0


In this chunk we:
- read the data from the migrants file
- get the migrants from the chosen country (to other countries) in the stated year 
- get the migrants from the chosen country (to the other countries) in the all previous years (from 1990 to the stated year)
- merge all this series into one dataframe

In [11]:
# get migrants from this COUNTRY in chosen YEAR 
participants = extract_participants('ev_all_votes.csv')
df = pd.read_csv('migrants.csv', dtype=str)
df = clean_data(participants, df)

# migrants in YEAR
migrants_from = get_migrants_from_country(df, str(COUNTRY.lower()), str(YEAR))

migrants = pd.DataFrame(list(migrants_from.items()), \
                        index = [x for x in range(len(migrants_from.items()))], \
                        columns = ['country', 'migrants'])
migrants[f'migrants{YEAR}'] = migrants['migrants'].apply(lambda x: x.replace(',', '')).astype(int)

# migrants in previous years
migrants_list = []
for year_number in range(1990, int(YEAR) + 1):
    migrants_dict = get_migrants_from_country(df, str(COUNTRY.lower()), str(year_number))
    # print(country_dict)
    if len(migrants_dict) != 0:
        migrants_year = pd.DataFrame(list(migrants_dict.items()), \
                        index = [x for x in range(len(migrants_dict.items()))], \
                        columns = ['country', f'migrants{year_number}'])
        # print(votes_year)
        migrants_year[f'migrants{year_number}'] = migrants_year[f'migrants{year_number}'].apply(lambda x: x.replace(',', '')).astype(int)
        # votes = pd.concat([votes, votes_year])
        # votes = votes.groupby(['country']).sum()
        # votes = pd.merge(votes, votes_year, on=['country'])
        migrants_list.append(migrants_year)

migrants_df = migrants_list[0].merge(migrants_list[1], on=['country'])
for year_num in range(2, int(YEAR) - 1990):
    migrants_df = migrants_df.merge(migrants_list[year_num], on=['country'])

migrants_df.head()

,country,migrants1990,migrants1991,migrants1992,migrants1993,migrants1994,migrants1995,migrants1996,migrants1997,migrants1998,...,migrants2005,migrants2006,migrants2007,migrants2008,migrants2009,migrants2010,migrants2011,migrants2012,migrants2013,migrants2014
0,armenia,227,227,227,227,227,321,321,321,321,...,607,607,607,607,607,994,994,994,994,994
1,azerbaijan,1178,1178,1178,1178,1178,1149,1149,1149,1149,...,856,856,856,856,856,667,667,667,667,667
2,cyprus,1291,1291,1291,1291,1291,3124,3124,3124,3124,...,5120,5120,5120,5120,5120,5276,5276,5276,5276,5276
3,georgia,5694,5694,5694,5694,5694,6612,6612,6612,6612,...,8550,8550,8550,8550,8550,10050,10050,10050,10050,10050
4,israel,190,190,190,190,190,271,271,271,271,...,338,338,338,338,338,324,324,324,324,324


Here we create the dataframe with the data about the physical borders of the chosen country with the other participants of the contest.
- we have 1, when the chosen country have the border with some other country in the list, and 0 otherwise.

In [13]:
# create dataframe with this COUNTRYs borders
borders = pd.read_csv('borders.csv')
country_borders = pd.DataFrame()
country_borders['country'] = borders['Country']
country_borders['borders'] = borders[str(COUNTRY).lower()].astype(int)

country_borders.head()

,country,borders
0,albania,0
1,armenia,0
2,australia,0
3,austria,0
4,azerbaijan,0


In the output of the following chink we can see the dataframe of the language variable.
We have the language of the winning song (languege of the song with with the chosen country performed)
And for the other countries we have two options - 1 if the winnig songs language matches the official language of the country and 0 otherwise.

In [14]:
# dataframe for languages match
language = pd.DataFrame()
language['country'] = pd.read_csv('borders.csv')['Country']
language['lang'] = np.where(pd.read_csv('borders.csv')['official_language'] == LANGUAGE, 1, 0)

language.head()

,country,lang
0,albania,0
1,armenia,0
2,australia,1
3,austria,0
4,azerbaijan,0


Here we merge all previousty obtained dataframes into one for the further analysis.
We merge these dataframes by the country column to get all observations for the same countries.

In [15]:
# merge votes, migrants and borders into one dataframe
votes_migrants = pd.merge(votes, migrants_df, on=['country'])
country_merged = votes_migrants.merge(country_borders, on=['country'])
merged_df = country_merged.merge(language, on=['country'])

merged_df.head()

,country,votes_this,votes_previous,migrants1990,migrants1991,migrants1992,migrants1993,migrants1994,migrants1995,migrants1996,...,migrants2007,migrants2008,migrants2009,migrants2010,migrants2011,migrants2012,migrants2013,migrants2014,borders,lang
0,albania,7.0,0.0,146,146,146,146,146,929,929,...,1247,1247,1247,709,709,709,709,709,0,0
1,armenia,7.0,0.0,227,227,227,227,227,321,321,...,607,607,607,994,994,994,994,994,0,0
2,australia,12.0,0.0,262,262,262,262,262,653,653,...,1289,1289,1289,1534,1534,1534,1534,1534,0,1
3,azerbaijan,6.0,0.0,1178,1178,1178,1178,1178,1149,1149,...,856,856,856,667,667,667,667,667,0,0
4,cyprus,10.0,0.0,1291,1291,1291,1291,1291,3124,3124,...,5120,5120,5120,5276,5276,5276,5276,5276,0,0


### Model 1
In this model we take:
- dependent valiable: votes in the chosen year
- independent variables:    
    - physical borders of the chosen country with other ones
    - match (or no match) in the language
    - migrants data from the chosen country to other ones from the 1990 to the chosen year

In [16]:
# Regression by migrants  

str_formula = f'votes_previous ~ borders + lang +'
for year in range(1990, int(YEAR) - 1):
    str_formula = str_formula + f'migrants{year} + '
str_formula = str_formula[:-2]

# model the Ordinary Least Squares regression
model = smf.ols(formula = str_formula, data=merged_df.dropna())
model = model.fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         votes_previous   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                 -0.218
Method:                 Least Squares   F-statistic:                    0.3869
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.897
Time:                        21:15:54   Log-Likelihood:                -71.721
No. Observations:                  25   AIC:                             159.4
Df Residuals:                      17   BIC:                             169.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.7361      1.610      2.942      0.009       1.339       8.133
borders          1.2719      2.466      0.516      0.613      -3.932       6.476
lang            -4.2243      5.361     -0.788      0.442     -15.535       7.087
migrants1990     0.0002      0.000      0.515      0.613      -0.001       0.001
migrants1991     0.0002      0.000      0.515      0.613      -0.001       0.001
migrants1992     0.0002      0.000      0.515      0.613      -0.001       0.001
migrants1993     0.0002      0.000      0.515      0.613      -0.001       0.001
migrants1994     0.0002      0.000      0.515      0.613      -0.001       0.001
migrants1995    -0.0005      0.001     -0.778      0.447      -0.002       0.001
migrants1996    -0.0005      0.001     -0.778      0.447      -0.002       0.001
migrants1997    -0.0005      0.001     -0.778      0.447      -0.002       0.001
migrants1998    -0.0005      0.001     -0.778      0.447      -0.002       0.001
migrants1999    -0.0005      0.001     -0.778      0.447      -0.002       0.001
migrants2000     0.0004      0.000      1.125      0.276      -0.000       0.001
migrants2001     0.0004      0.000      1.125      0.276      -0.000       0.001
migrants2002     0.0004      0.000      1.125      0.276      -0.000       0.001
migrants2003     0.0004      0.000      1.125      0.276      -0.000       0.001
migrants2004     0.0004      0.000      1.125      0.276      -0.000       0.001
migrants2005    -0.0002      0.000     -0.647      0.526      -0.001       0.000
migrants2006    -0.0002      0.000     -0.647      0.526      -0.001       0.000
migrants2007    -0.0002      0.000     -0.647      0.526      -0.001       0.000
migrants2008    -0.0002      0.000     -0.647      0.526      -0.001       0.000
migrants2009    -0.0002      0.000     -0.647      0.526      -0.001       0.000
migrants2010 -4.686e-05   4.81e-05     -0.975      0.343      -0.000    5.46e-05
migrants2011 -4.686e-05   4.81e-05     -0.975      0.343      -0.000    5.46e-05
migrants2012 -4.686e-05   4.81e-05     -0.975      0.343      -0.000    5.46e-05
migrants2013 -4.686e-05   4.81e-05     -0.975      0.343      -0.000    5.46e-05
==============================================================================
Omnibus:                        4.707   Durbin-Watson:                   1.743
Prob(Omnibus):                  0.095   Jarque-Bera (JB):                2.057
Skew:                           0.373   Prob(JB):                        0.358
Kurtosis:                       1.809   Cond. No.                     3.42e+66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The input rank is higher than the number of observations.
[3] The smallest 

In the following chunk we:
- form the dataframe withh all the votes given to the chosen country from the other ones from 1990 to the chosen year

In [17]:
votes_list = []
for year_number in range(1990, int(YEAR)):
    country_dict = get_votes_from_country(str(COUNTRY.lower()), 'televoters', str(year_number))
    # print(country_dict)
    if len(country_dict) != 0:
        votes_year = pd.DataFrame(list(country_dict.items()), \
                        index = [x for x in range(len(country_dict.items()))], \
                        columns = ['country', f'votes{year_number}'])
        votes_year[f'votes{year_number}'] = votes_year[f'votes{year_number}'].astype(int)
        votes_list.append(votes_year)

# dataframe with votes from 1990 to the YEAR
votes_df = pd.concat([votes_list[0], votes_list[1]])
for year_num in range(2, int(YEAR) - 1990 - 1):
    votes_df = pd.concat([votes_df, votes_list[year_num]])
    votes_df = votes_df.groupby(['country']).sum().reset_index()

votes_df = votes_df.merge(votes, on=['country'])

votes_df.head()

,country,votes1990,votes1991,votes1992,votes1993,votes1994,votes1995,votes1996,votes1997,votes1998,...,votes2006,votes2007,votes2008,votes2009,votes2011,votes2012,votes2013,votes2014,votes_this,votes_previous
0,albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,0.0,5.0,0.0,0.0,7.0,0.0
1,armenia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
2,austria,0.0,10.0,0.0,10.0,5.0,8.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,6.0
3,azerbaijan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,7.0,0.0,0.0,6.0,0.0
4,belgium,0.0,10.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,0.0


Here we merge all previousty obtained dataframes into one for the further analysis.
We merge these dataframes by the country column to get all observations for the same countries.

In [19]:
country_merged = votes_df.merge(country_borders, on=['country'])
merged_df = country_merged.merge(language, on=['country'])
merged_df.head()

,country,votes1990,votes1991,votes1992,votes1993,votes1994,votes1995,votes1996,votes1997,votes1998,...,votes2008,votes2009,votes2011,votes2012,votes2013,votes2014,votes_this,votes_previous,borders,lang
0,albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,5.0,0.0,0.0,7.0,0.0,0,0
1,armenia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0,0
2,austria,0.0,10.0,0.0,10.0,5.0,8.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,7.0,6.0,0,0
3,azerbaijan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,7.0,0.0,0.0,6.0,0.0,0,0
4,belgium,0.0,10.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,12.0,0.0,0,0


### Model 2
In this model we take:
- dependent valiable: votes in the chosen year
- independent variables:    
    - physical borders of the chosen country with other ones
    - match (or no match) in the language
    - votes data (votes given to the chosen country by other participants from the 1990 to the stated year)

In [20]:
str_formula = f'votes_previous ~ borders + lang +'
for year in range(1990, int(YEAR) - 1):
    str_formula = str_formula + f'votes{year} + '
str_formula = str_formula[:-2]
str_formula = str_formula.replace('votes2010 +', '')

# model the Ordinary Least Squares regression
model_2 = smf.ols(formula = str_formula, data=merged_df.dropna())
model_2 = model_2.fit()

model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         votes_previous   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     1.589
Date:                Wed, 01 Jun 2022   Prob (F-statistic):              0.397
Time:                        21:16:35   Log-Likelihood:                -44.989
No. Observations:                  29   AIC:                             142.0
Df Residuals:                       3   BIC:                             177.5
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3780      2.298      0.600      0.591      -5.935       8.691
borders      -34.4592     30.822     -1.118      0.345    -132.550      63.631
lang           6.5918     15.160      0.435      0.693     -41.653      54.836
votes1990      2.4711      4.076      0.606      0.587     -10.500      15.442
votes1991     -0.9272      1.126     -0.823      0.471      -4.511       2.657
votes1992      2.3267      9.586      0.243      0.824     -28.180      32.833
votes1993      0.4799      1.035      0.464      0.674      -2.814       3.774
votes1994     -1.9943      2.939     -0.679      0.546     -11.348       7.359
votes1995      1.4754      1.631      0.905      0.432      -3.715       6.666
votes1996      1.6800      1.757      0.956      0.409      -3.911       7.271
votes1997      1.2280      2.083      0.589      0.597      -5.402       7.858
votes1998      2.6113      2.590      1.008      0.388      -5.630      10.853
votes1999     -3.8810      3.478     -1.116      0.346     -14.949       7.187
votes2000      1.2180      2.027      0.601      0.590      -5.234       7.670
votes2001      3.9041      3.995      0.977      0.401      -8.811      16.619
votes2002      3.4574      3.801      0.910      0.430      -8.638      15.553
votes2003     -0.7388      1.307     -0.565      0.611      -4.898       3.421
votes2004      2.3129      1.983      1.167      0.328      -3.996       8.622
votes2005      0.7129      5.718      0.125      0.909     -17.483      18.909
votes2006      0.1642      0.672      0.244      0.823      -1.975       2.304
votes2007      2.8209      2.163      1.304      0.283      -4.061       9.703
votes2008     -4.9459      4.037     -1.225      0.308     -17.795       7.903
votes2009     -1.3793      3.179     -0.434      0.694     -11.497       8.738
votes2011     -2.3548      2.673     -0.881      0.443     -10.861       6.152
votes2012      1.1155      0.983      1.135      0.339      -2.013       4.244
votes2013      2.5252      3.776      0.669      0.551      -9.491      14.541
==============================================================================
Omnibus:                       21.778   Durbin-Watson:                   1.525
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.423
Skew:                           1.406   Prob(JB):                     1.85e-11
Kurtosis:                       8.744   Cond. No.                         730.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""